# Hello Bluesky: Reading detectors and scanning

In this notebook you will:

* Connect to some simulated hardware.
* Acquire some data via two common experimental procedures ("plans"), ``count`` and ``scan``.
* Write a custom plan.

We will use the following modules---please import them before continuing

## Configuration

Run the following cell to ensure that all of the yaq daemons are running properly. Everything should be online and not busy.

In [ ]:
%%bash
yaqd status

Now, let's generate yaqc-bluesky device objects for each of our daemons.

In [ ]:
import yaqc_bluesky
d1 = yaqc_bluesky.Device(38100)

## Data Acquisition

### Executing a `count` plan with various parameters

In [ ]:
from bluesky.plans import count
RE(count([det]))